In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df=pd.read_pickle('wafer_map.pkl')
df

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType,failureNum
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,1.0,[[Training]],[[none]],8
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,2.0,[[Training]],[[none]],8
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,3.0,[[Training]],[[none]],8
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,4.0,[[Training]],[[none]],8
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,5.0,[[Training]],[[none]],8
...,...,...,...,...,...,...,...
172945,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2,...",600.0,lot47542,21.0,[[Test]],[[Edge-Loc]],2
172946,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,22.0,[[Test]],[[Edge-Loc]],2
172947,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1,...",600.0,lot47542,23.0,[[Test]],[[Edge-Ring]],3
172948,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1,...",600.0,lot47542,24.0,[[Test]],[[Edge-Loc]],2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172950 entries, 0 to 172949
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   waferMap        172950 non-null  object 
 1   dieSize         172950 non-null  float64
 2   lotName         172950 non-null  object 
 3   waferIndex      172950 non-null  float64
 4   trianTestLabel  172950 non-null  object 
 5   failureType     172950 non-null  object 
 6   failureNum      172950 non-null  object 
dtypes: float64(2), object(5)
memory usage: 9.2+ MB


In [4]:
df_withpattern = df[(df['failureNum']>=0) & (df['failureNum']<=7)]
df_withpattern = df_withpattern.reset_index()

## Feature extraction

### Density based feature

divide Wafer map into 13 regions and computed defects density accordingly. The 13 regions includes the inner 9 same regions and the top, bottom, left and right regions.

In [5]:
def cal_den(x):
    return 100*(np.sum(x==2)/np.size(x))  
def find_regions(x):
    rows=np.size(x,axis=0)
    cols=np.size(x,axis=1)
    ind1=np.arange(0,rows,rows//5)
    ind2=np.arange(0,cols,cols//5)
    
    reg1=x[ind1[0]:ind1[1],:]
    reg3=x[ind1[4]:,:]
    reg4=x[:,ind2[0]:ind2[1]]
    reg2=x[:,ind2[4]:]

    reg5=x[ind1[1]:ind1[2],ind2[1]:ind2[2]]
    reg6=x[ind1[1]:ind1[2],ind2[2]:ind2[3]]
    reg7=x[ind1[1]:ind1[2],ind2[3]:ind2[4]]
    reg8=x[ind1[2]:ind1[3],ind2[1]:ind2[2]]
    reg9=x[ind1[2]:ind1[3],ind2[2]:ind2[3]]
    reg10=x[ind1[2]:ind1[3],ind2[3]:ind2[4]]
    reg11=x[ind1[3]:ind1[4],ind2[1]:ind2[2]]
    reg12=x[ind1[3]:ind1[4],ind2[2]:ind2[3]]
    reg13=x[ind1[3]:ind1[4],ind2[3]:ind2[4]]
    fea_reg_den=[]
    fea_reg_den =[cal_den(reg1),cal_den(reg2),cal_den(reg3),cal_den(reg4),cal_den(reg5),cal_den(reg6),cal_den(reg7),cal_den(reg8),cal_den(reg9),cal_den(reg10),cal_den(reg11),cal_den(reg12),cal_den(reg13)]
    return fea_reg_den

In [6]:
df_withpattern['fea_reg']=df_withpattern.waferMap.apply(find_regions)

In [7]:
df_withpattern_copy = df_withpattern.copy()

In [8]:
df_all=df_withpattern_copy.copy()
a=[df_all['fea_reg'][i] for i in range(df_all.shape[0])]
fea_all =  np.array(a)

In [9]:
label=[df_all.failureNum[i] for i in range(df_all.shape[0])]
label=np.array(label)

In [10]:
x = fea_all
y = label

## Test split validation
**find the optimal test_size**

In [11]:
lst_test_size=[0.3,0.25,0.2]
for testsize in lst_test_size:
    x_train,x_test, y_train,  y_test=train_test_split(x,y, test_size=testsize,random_state=42)
    
    model= LogisticRegression(multi_class='multinomial')
    model.fit(x_train, y_train)
    
    score_train=model.score(x_train, y_train)
    score_test=model.score(x_test, y_test)
    print('with(',(1-testsize),':',testsize,'),score train: ',round(score_train,4),', score test: ',round(score_test,4),', diff: ',round(abs(score_train-score_test),4))

C:\Users\minhh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


with( 0.7 : 0.3 ),score train:  0.703 , score test:  0.7026 , diff:  0.0004


C:\Users\minhh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


with( 0.75 : 0.25 ),score train:  0.7046 , score test:  0.7061 , diff:  0.0015
with( 0.8 : 0.2 ),score train:  0.7053 , score test:  0.711 , diff:  0.0057


C:\Users\minhh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The test_size = 0.2 has the highest accuracy_score in both train and test. Thus we choose test_size = 0.2 as an optimal test_size

## LogisticRegression model

In [12]:

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=42)  

In [13]:

model=LogisticRegression(multi_class='multinomial')
model.fit(x_train, y_train)

C:\Users\minhh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [14]:
y_pred=model.predict(x_test)

In [15]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [16]:
print('accuracy: ',accuracy_score(y_test, y_pred))

accuracy:  0.7110109717868338


In [17]:
print(confusion_matrix(y_test,y_pred))

[[ 724    7    9    1   79    0    0    0]
 [   3   88    1    0   26    0    0    1]
 [  15    2  540  343  142   29    0    0]
 [   0    0  126 1757   57    0    0    0]
 [  76   23  160   31  424    0    0    1]
 [   7    1   44    0   35   95    0    0]
 [  10    1   37    9  175    0    0    0]
 [   0    0    0    0    0   24    0    1]]


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.88      0.87       820
           1       0.72      0.74      0.73       119
           2       0.59      0.50      0.54      1071
           3       0.82      0.91      0.86      1940
           4       0.45      0.59      0.51       715
           5       0.64      0.52      0.58       182
           6       0.00      0.00      0.00       232
           7       0.33      0.04      0.07        25

    accuracy                           0.71      5104
   macro avg       0.55      0.52      0.52      5104
weighted avg       0.68      0.71      0.69      5104



C:\Users\minhh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\minhh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\minhh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo